In [2]:
import pandas as pd

In [3]:
import numpy as np

### Carregar dados

In [4]:
jogos = pd.read_csv("data/jogos.csv")

In [5]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante
0,2017,Vasco,Coritiba,1,1
1,2017,Flamengo,Bahia,4,1
2,2017,Sport Recife,Santos,1,1
3,2017,Palmeiras,Ponte Preta,2,0
4,2017,Vitória,Atlético-PR,2,3


In [6]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 5 columns):
Ano                 1431 non-null int64
Mandante            1431 non-null object
Visitante           1431 non-null object
Placar_mandante     1431 non-null int64
Placar_visitante    1431 non-null int64
dtypes: int64(3), object(2)
memory usage: 56.0+ KB


### Remover espaços em branco dos nomes dos times

In [7]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [8]:
times = list(jogos["Visitante"].unique())

In [9]:
times.sort()

In [10]:
times

['América-MG',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Botafogo',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Goias',
 'Grêmio',
 'Internacional',
 'Joinville',
 'Palmeiras',
 'Ponte Preta',
 'Santa Cruz',
 'Santos',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [17]:
len(times)

27

### Criar coluna com resultado

In [18]:
def resultado(row):
    if (row["Placar_mandante"] > row["Placar_visitante"]):
        return "MANDANTE"
    elif (row["Placar_mandante"] < row["Placar_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [19]:
jogos['Resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [21]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante,Resultado
0,2017,Vasco,Coritiba,1,1,EMPATE
1,2017,Flamengo,Bahia,4,1,MANDANTE
2,2017,Sport Recife,Santos,1,1,EMPATE
3,2017,Palmeiras,Ponte Preta,2,0,MANDANTE
4,2017,Vitória,Atlético-PR,2,3,VISITANTE
